# LangChain Basics


In [3]:
#!pip install -r ./requirements.txt -q

In [4]:
#!pip show langchain

In [5]:
#!pip install langchain --upgrade -q

### Python-dotenv


In [6]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(),override=True)

#os.environ.get('PINECONE_API_KEY')

True

### LLM Models (Wrappers): GPT-3

In [7]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=0.7, max_tokens=512)
print(llm)

OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.7, 'max_tokens': 512, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}}


In [8]:
output = llm('Explain quantum mechanics in one sentence')
print(output)



Quantum mechanics is a set of principles describing the behavior of matter and energy at the atomic and subatomic scales.


In [9]:
print(llm.get_num_tokens('Explain quantum mechanics in one sentence'))

7


In [10]:
output = llm.generate(['... is the capital of France', 
                   'What is the formula for the area of the circle'])
print(output.generations)

[[Generation(text='\nParis', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nThe formula for the area of a circle is A = πr², where A is the area, π is 3.14, and r is the radius of the circle.', generation_info={'finish_reason': 'stop', 'logprobs': None})]]


In [11]:
# Answer of the second question
print(output.generations[1][0].text)



The formula for the area of a circle is A = πr², where A is the area, π is 3.14, and r is the radius of the circle.


In [12]:
len(output.generations)

2

In [13]:
output = llm.generate(['Write an original tagline for burger restaurant']*3)

In [14]:
for o in output.generations:
    print(o[0].text)



"Taste the flavor of freshness in every bite!"


"Tastes good, feels good - burgers at their best!"


"Taste the Juicy Difference at Our Burger Joint!"


### ChatModels: GPT-3.5-Turbo and GPT-4

In [15]:
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI

In [16]:
chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5, max_tokens=1024)
messages = [
    SystemMessage(content='You are a physicist and respond only in Spanish'),
    HumanMessage(content='Explain quantum mechanics in one sentence')
]
output = chat(messages)

In [17]:
print(output)

content='La mecánica cuántica es una teoría física que describe el comportamiento de las partículas subatómicas, las cuales pueden existir en múltiples estados al mismo tiempo y solo se determinan al ser observadas.'


### Prompt Templates

In [18]:
from langchain import PromptTemplate

In [19]:
template = '''You are an experienced virologist. Write a few sentences about the following {virus} in {language}'''
prompt = PromptTemplate(
    input_variables=['virus','language'],
    template=template
)
print(prompt)

input_variables=['virus', 'language'] template='You are an experienced virologist. Write a few sentences about the following {virus} in {language}'


In [20]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=0.7)
output = llm(prompt.format(virus='ebola', language='spanish'))

In [21]:
print(output)



Como virologista experimentado, sé que el ébola es un virus letal que puede causar enfermedades graves y a veces letales. El ébola se transmite principalmente a través de contacto directo con los fluidos corporales de una persona infectada. Los síntomas pueden incluir fiebre, dolor de cabeza, dolor muscular y fatiga. La única forma de prevenir la propagación del ébola es mediante la implementación de prácticas de higiene adecuadas y el aislamiento de los individuos infectados.


### Simple Chains

In [22]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)

template = '''You are an experienced virologist. Write a few sentences about the following {virus} in {language}'''

prompt = PromptTemplate(
    input_variables=['virus','language'],
    template=template
)

chain = LLMChain(llm=llm, prompt=prompt)
output = chain.run({'virus':'HSV', 'language':'spanish'})

In [26]:
print(output)

El virus del herpes simple (HSV, por sus siglas en inglés) es un virus altamente contagioso que afecta principalmente a los humanos. Hay dos tipos de HSV: el HSV-1, que suele causar infecciones orales como el herpes labial, y el HSV-2, que se asocia comúnmente con las infecciones genitales. Ambos tipos de HSV pueden causar brotes recurrentes a lo largo de la vida de una persona infectada. Actualmente, no existe una cura para el HSV, pero los antivirales pueden ayudar a controlar los síntomas y reducir la frecuencia de los brotes.


### Sequencial Chains

In [27]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

llm1 = OpenAI(model_name='text-davinci-003', temperature=0.7, max_tokens=1024)
prompt1 = PromptTemplate(
    input_variables=['concept'],
    template='''You are an expereinced scientist and Python programmer. 
        Write a function that implements the concept of {concept}.'''
)
chain1 = LLMChain(llm=llm1, prompt=prompt1)


llm2 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1.2)
prompt2 = PromptTemplate(
    input_variables=['function'],
    template='Given the Python function {function}, describe it as detailed as possible.'
)
chain2 = LLMChain(llm=llm2, prompt=prompt2)

overall_chain = SimpleSequentialChain(chains=[chain1,chain2], verbose=True)
output = overall_chain.run('linear regression')



> Entering new SimpleSequentialChain chain...


def linear_regression(x_values, y_values):
    """
    Implements the concept of linear regression
    :param x_values: List of x values
    :param y_values: List of y values
    :return: A list of coefficients for the linear regression
    """
    x_mean = sum(x_values) / len(x_values)
    y_mean = sum(y_values) / len(y_values)

    numerator = sum(x*y for x, y in zip(x_values, y_values)) - len(x_values) * x_mean * y_mean
    denominator = sum(x**2 for x in x_values) - len(x_values) * x_mean**2

    slope = numerator / denominator
    intercept = y_mean - (slope * x_mean)

    return [intercept, slope]
The function `linear_regression` implements the concept of linear regression. Linear regression is a statistical technique used to find the best-fitting line that describes the relationship between two variables. In this case, the function takes in a list of x values and a list of corresponding y values.

The function starts by calculati

### LangChain Agents

In [28]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.llms import OpenAI

In [29]:
llm = OpenAI(temperature=0)
agent_executer = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)
agent_executer.run('Calculate the squate root of the factorial fo 20 \
                   and display it with 4 decimal points')



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


 I need to calculate the factorial of 20 and then take the square root
Action: Python_REPL
Action Input: from math import factorial
               print(round(factorial(20)**0.5, 4))
Observation: IndentationError('unexpected indent', ('<string>', 2, 15, '               print(round(factorial(20)**0.5, 4))\n', 2, -1))
Thought: I need to remove the extra indentation
Action: Python_REPL
Action Input: from math import factorial
print(round(factorial(20)**0.5, 4))
Observation: 1559776268.6285

Thought: I now know the final answer
Final Answer: 1559776268.6285

> Finished chain.


'1559776268.6285'

In [30]:
agent_executer.run('What is the answer to 5.1 ** 7.3')



> Entering new AgentExecutor chain...
 I need to use the Python REPL to calculate this
Action: Python_REPL
Action Input: print(5.1 ** 7.3)
Observation: 146306.05007233328

Thought: I now know the final answer
Final Answer: 146306.05007233328

> Finished chain.


'146306.05007233328'